In [1]:
import requests 
import datetime as dt
import re 
import pandas as pd 
import time 
import json                                                                               
import urllib.request #
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from datetime import datetime            
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome('./chromedriver.exe')              # chrome창을 띄움, Mac은 .exe 빼고 실행
driver.implicitly_wait(2)
driver.get("https://www.youtube.com/")                                                                     # youtube창을 띄움
driver.implicitly_wait(2)
# driver.maximize_window()
driver.execute_script("window.scrollTo(0, window.scrollY + 300)")

In [3]:
# 페이지를 내리는 함수를 만듬
def pagedown(num):
    num_of_pagedowns = num 
    body = driver.find_element_by_tag_name('body')
    while num_of_pagedowns:
        body.send_keys(Keys.END)
        time.sleep(2)     
        num_of_pagedowns -= 1

In [4]:
def ScrollDowntoOneYear():
    driver.set_window_size(930,700)
    body = driver.find_element_by_tag_name('body')
    n = 0
    while True:
        body.send_keys(Keys.END)
        time.sleep(2)
        page = driver.page_source
        soup = bs(page, 'lxml')
        all_videos = soup.find_all(id='dismissable')
        for i in all_videos:
            texts = i.find('div',{'id':'metadata'}).find_all('span')
            for j in texts:
                if re.findall("1년 전", j.text) == ['1년 전']:
                    return soup.find_all(id='dismissable')
                elif re.findall("2년 전", j.text) == ['2년 전']:
                    return soup.find_all(id='dismissable')

In [5]:
def GetVideoUrls(all_videos):
    video_url = []
    for i in all_videos:
        texts = i.find('div',{'id':'metadata'}).find_all('span')
        for j in texts:
            if re.findall("1년 전", j.text) == ['1년 전']:
                return video_url
            if re.findall("2년 전", j.text) == ['2년 전']:
                return video_url
        url = base_url + i.find('a',{'id':'thumbnail'})['href']
        video_url.append(url)

In [18]:
def GetComment():
    try:
        comment = driver.find_element_by_xpath('//*[@id="count"]/yt-formatted-string').text
        return comment
    except:
        return 'NaN'
    
def GetLikes():
    try:
        sourcecode = driver.page_source
        html = bs(sourcecode, 'html.parser')
        likes_num = html.find('yt-formatted-string',{'id':"text",'class':"style-scope ytd-toggle-button-renderer style-text",'aria-label':re.compile('좋아요 ')})['aria-label']
        return likes_num
    except:
        return 'NaN'
    
def GetDislikes():
    try:
        sourcecode = driver.page_source
        html = bs(sourcecode, 'html.parser')
        unlikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요 ')})['aria-label']
        return unlikes_num
    except:
        return 'NaN'

In [19]:
def CreateVideosInfoData(video_url, title_data, i):
    for k in video_url:
        driver.get(k)
        pagedown(3)
        time.sleep(2)

        title = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text    
        view = driver.find_element_by_css_selector('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer').text
        upload_date = driver.find_element_by_css_selector('#date > yt-formatted-string').text

        comment = GetComment()

        likes_num = GetLikes()
        #싫어요수t
        unlikes_num = GetDislikes()

        insert_data = pd.DataFrame({'title_list':[title],
                                    'view':[view],
                                    'upload_date':[upload_date],
                                    'comment':[comment],
                                    'likes_num':[likes_num],
                                    'unlikes_num':[unlikes_num]})

        title_data = title_data.append(insert_data)
    
    title_data.to_csv(path_or_buf = f'./{i}.csv',
                      header = True,
                      index = False,
                      encoding = 'utf-8-sig')

In [28]:
channels = ['LG전자 LG Electronics korea','ncsoft',
    '현대자동차'] # 자신이 맡은 채널 리스트 안에 넗고 실행

for i in channels:
    
    title_data = pd.DataFrame({'title_list':[],
                               'view':[],
                               'upload_date':[],
                               'comment':[],
                               'likes_num':[],
                               'unlikes_num':[]})
    
    base_url = 'http://www.youtube.com'
    
    search = driver.find_element_by_name("search_query")                                                    
    search.clear()                                                                                          
    search.send_keys(i)
    search.submit()
    driver.find_element_by_id("avatar").click()
    time.sleep(1.5)
    driver.find_element_by_xpath('//*[@id="tabsContent"]/paper-tab[2]/div').click()
    driver.get(driver.current_url)
    
    all_videos = ScrollDowntoOneYear()
    video_url = GetVideoUrls(all_videos)
    CreateVideosInfoData(video_url, title_data, i)
    